In [1]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

Using TensorFlow backend.


In [56]:
img = load_img('eeg_small.png')

In [57]:
print(type(img))
print(img.format)
print(img.mode)
print(img.size)

<class 'PIL.Image.Image'>
None
RGB
(432, 288)


In [58]:
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.Image.Image'>
float32
(288, 432, 3)
<class 'PIL.Image.Image'>


In [59]:
import os
import numpy as np
import pandas as pd
base_path = "spectrogram_images"
relaxed_images_df = pd.DataFrame()
for filename in os.listdir(base_path):
    filepath = os.path.join(base_path, filename)
    #img = load_img(filepath)
    #img_array = img_to_array(img)
    #relaxed_images_df.append(img_array)
print(relaxed_images_df.shape)

(0, 0)


In [2]:
datagen = ImageDataGenerator()
train_base_path = "spectrogram_images/train"
test_base_path = "spectrogram_images/test"
validation_base_path = "spectrogram_images/validation"
train_generator = datagen.flow_from_directory(
        train_base_path,
        target_size=(288, 432),
        class_mode='binary')
test_generator = datagen.flow_from_directory(
        test_base_path,
        target_size=(288, 432),
        class_mode='binary')
validation_generator = datagen.flow_from_directory(
        validation_base_path,
        target_size=(288, 432),
        class_mode='binary')

Found 1344 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [3]:
# confirm the iterator works
batchX, batchy = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
batchX, batchy = validation_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 288, 432, 3), min=0.000, max=255.000
Batch shape=(32, 288, 432, 3), min=0.000, max=255.000


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288, 432, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=42, validation_data=validation_generator, validation_steps=6)

Epoch 1/10
42/42 [==============================] - 195s 5s/step - loss: 7.8170 - acc: 0.5097 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
42/42 [==============================] - 250s 6s/step - loss: 8.2796 - acc: 0.4807 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
42/42 [==============================] - 206s 5s/step - loss: 7.8051 - acc: 0.5104 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
42/42 [==============================] - 203s 5s/step - loss: 7.8763 - acc: 0.5060 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/10
42/42 [==============================] - 196s 5s/step - loss: 8.1610 - acc: 0.4881 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/10
42/42 [==============================] - 217s 5s/step - loss: 7.7814 - acc: 0.5119 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 7/10
42/42 [==============================] - 236s 6s/step - loss: 8.1373 - acc: 0.4896 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 8/10
42/42 [==============================] - 203s 5s/step - loss: 7.8170 - a

In [39]:
# evaluate model
loss = model.evaluate_generator(test_generator, steps=6)

In [40]:
print(loss)

[8.346871058146158, 0.48214285714285715]
